Asociaedro


In [1]:
# 1) Definimos la variable simbólica
m, t = var('m t')

# 2) Implementamos la fórmula cerrada para H_{S_d}(m)
def Ehrhart_Stasheff_closed(d):
    d = Integer(d)
    H = sum( (1/(d+1)) * binomial(d+1, k+1) * binomial(d+1, k) * binomial(m + d - k, d)
             for k in range(d+1) )
    return expand(H)

# 3) Dada la serie h*(t) = ∑ h_i t^i sobre (1−t)^{d+1}, extraemos el h*–vector
def h_star_from_series(series_num, d):
    # series_num es el numerador ∑ h_i t^i
    return [ series_num.coefficient(t, i) for i in range(d+1) ]

# 4) Comprobación palindrómica de h* (criterio de reflexividad)
def es_reflexivo_por_hstar(hstar):
    return all(hstar[i] == hstar[-(i+1)] for i in range(len(hstar)))

# 5) Exploramos para d = 2,3,4
for d in [2,3,4]:
    print(f"--- Dimension {d} ---")
    # a) Polinomio de Ehrhart
    E = Ehrhart_Stasheff_closed(d)
    print("L_P(m) =", E)

    M = matrix(QQ, [[ binomial(m_val + d - i, d) 
                      for i in range(d+1)] 
                    for m_val in range(d+1)])
    vals = vector(QQ, [E(m=m_val) for m_val in range(d+1)])
    hstar = M.solve_right(vals)
    print("h*–vector:", list(hstar))
    # c) Serie de Ehrhart simbólica
    Ht = sum(hstar[i]*t^i for i in range(d+1))
    print("E_P(t) =", Ht, "/ (1 - t)^", d+1)
    # d) Reflexividad por palíndromo
    print("Reflexivo? palindrómico h* →", es_reflexivo_por_hstar(hstar))
    print()


--- Dimension 2 ---
L_P(m) = 5/2*m^2 + 5/2*m + 1


h*–vector: [1, 3, 1]
E_P(t) = t^2 + 3*t + 1 / (1 - t)^ 3
Reflexivo? palindrómico h* → True

--- Dimension 3 ---
L_P(m) = 7/3*m^3 + 7/2*m^2 + 19/6*m + 1
h*–vector: [1, 6, 6, 1]
E_P(t) = t^3 + 6*t^2 + 6*t + 1 / (1 - t)^ 4
Reflexivo? palindrómico h* → True

--- Dimension 4 ---
L_P(m) = 7/4*m^4 + 7/2*m^3 + 21/4*m^2 + 7/2*m + 1
h*–vector: [1, 10, 20, 10, 1]
E_P(t) = t^4 + 10*t^3 + 20*t^2 + 10*t + 1 / (1 - t)^ 5
Reflexivo? palindrómico h* → True



In [2]:

# 1) Construcción de S_d = Conv({±e_i : 1≤i≤d} ∪ {e_i + … + e_j : 1≤i<j≤d})
def Stasheff_dual(d):
    d = Integer(d)
    verts = []
    # ±e_i
    for i in range(d):
        ei = vector(QQ, [1 if k == i else 0 for k in range(d)])
        verts.append(ei)
        verts.append(-ei)
    # e_i + … + e_j, con 1 ≤ i < j ≤ d
    for i in range(d):
        for j in range(i+1, d):
            v = vector(QQ, [1 if i <= k <= j else 0 for k in range(d)])
            verts.append(v)
    return Polyhedron(vertices=verts)

# 2) El associaedro A_d es simplemente el polar de S_d
def associahedron(d):
    Sd = Stasheff_dual(d)
    return Sd.polar()

# 3) Ejemplo de uso, sin gráficas
for d in [2,3,4]:
    Sd = Stasheff_dual(d)
    Ad = associahedron(d)
    print(f"d = {d}: S_d tiene {len(Sd.vertices_list())} vértices; "
          f"el associaedro A_d = S_d^◦ tiene {len(Ad.vertices_list())} vértices.")


d = 2: S_d tiene 5 vértices; el associaedro A_d = S_d^◦ tiene 5 vértices.
d = 3: S_d tiene 9 vértices; el associaedro A_d = S_d^◦ tiene 12 vértices.
d = 4: S_d tiene 14 vértices; el associaedro A_d = S_d^◦ tiene 28 vértices.


In [3]:
###############################################################
#  Ehrhart de los politopos de Stasheff mediante la recurrencia (3.1)
#  compatible con SageMath ≥ 10.6   (sin Normaliz ni LattE)
###############################################################
from sage.all import QQ, PolynomialRing, matrix, vector, var

# Anillo Q[n]
R.<n> = PolynomialRing(QQ)

# -----------------------------------------------------------------
# 1.  Recurrencia  E_d = (2n+1)E_{d-1} – ½ n(n+1)E_{d-2}
# -----------------------------------------------------------------
def ehrhart_stasheff_recurrence(d):
    if d < 0:
        raise ValueError("d debe ser ≥ 0")

    E0 = R(1)
    E1 = 2*n + 1
    if d == 0:
        return E0
    if d == 1:
        return E1

    E_prev2, E_prev1 = E0, E1
    for _ in range(2, d + 1):
        E_k = (2*n + 1) * E_prev1 - QQ(1)/2 * n * (n + 1) * E_prev2
        E_prev2, E_prev1 = E_prev1, R(E_k)      # asegura que está en R
    return E_prev1

# -----------------------------------------------------------------
# 2.  Descomposición mágica  E(n) = Σ a_j n^j (1+n)^{d-j}
# -----------------------------------------------------------------
def magic_decomposition(poly, d):
    """
    Devuelve los coeficientes a_j tales que
        poly(n) = Σ_{j=0}^d  a_j n^j (1+n)^{d-j}.
    """
    # Matriz de cambio de base   B_{ij} = coef. de n^i en n^j(1+n)^{d-j}
    B = matrix(QQ, d + 1, d + 1,
               lambda i, j: R((n**j) * (1 + n)**(d - j))[i])
    b = vector(QQ, [poly[i] for i in range(d + 1)])   # coef. de poly
    return list(B.solve_right(b))                     # a = B^{-1} b

# -----------------------------------------------------------------
# 3.  Demostración rápida  (d = 2,3,4,5)
# -----------------------------------------------------------------
if __name__ == "__main__":
    for d in [2, 3, 4, 5, 6]:
        E = ehrhart_stasheff_recurrence(d)
        a = magic_decomposition(E, d)

        print(f"\nStasheff  d = {d}")
        print("E_{St_d}(n) =", E)
        print("Forma mágica:",
              " + ".join(f"{ai}·n^{j}(1+n)^{d-j}"
                         for j, ai in enumerate(a) if ai))



Stasheff  d = 2
E_{St_d}(n) = 7/2*n^2 + 7/2*n + 1
Forma mágica: 1·n^0(1+n)^2 + 3/2·n^1(1+n)^1 + 1·n^2(1+n)^0

Stasheff  d = 3
E_{St_d}(n) = 6*n^3 + 9*n^2 + 5*n + 1
Forma mágica: 1·n^0(1+n)^3 + 2·n^1(1+n)^2 + 2·n^2(1+n)^1 + 1·n^3(1+n)^0

Stasheff  d = 4
E_{St_d}(n) = 41/4*n^4 + 41/2*n^3 + 67/4*n^2 + 13/2*n + 1
Forma mágica: 1·n^0(1+n)^4 + 5/2·n^1(1+n)^3 + 13/4·n^2(1+n)^2 + 5/2·n^3(1+n)^1 + 1·n^4(1+n)^0

Stasheff  d = 5
E_{St_d}(n) = 35/2*n^5 + 175/4*n^4 + 47*n^3 + 107/4*n^2 + 8*n + 1
Forma mágica: 1·n^0(1+n)^5 + 3·n^1(1+n)^4 + 19/4·n^2(1+n)^3 + 19/4·n^3(1+n)^2 + 3·n^4(1+n)^1 + 1·n^5(1+n)^0

Stasheff  d = 6
E_{St_d}(n) = 239/8*n^6 + 717/8*n^5 + 953/8*n^4 + 711/8*n^3 + 39*n^2 + 19/2*n + 1
Forma mágica: 1·n^0(1+n)^6 + 7/2·n^1(1+n)^5 + 13/2·n^2(1+n)^4 + 63/8·n^3(1+n)^3 + 13/2·n^4(1+n)^2 + 7/2·n^5(1+n)^1 + 1·n^6(1+n)^0


In [1]:
from sage.all import QQ, PolynomialRing, binomial, var, matrix, vector
R.<n> = PolynomialRing(QQ)

def ehrhart_stasheff(d):
 
    if d < 0:
        raise ValueError("d debe ser ≥ 0")

    E0 = R(1)
    E1 = 2*n + 1
    if d == 0:
        return E0
    if d == 1:
        return E1

    E_prev2, E_prev1 = E0, E1
    for _ in range(2, d + 1):
        E_k = (2*n + 1)*E_prev1 - QQ(1)/2 * n * (n + 1) * E_prev2
        E_prev2, E_prev1 = E_prev1, R(E_k)
    return E_prev1

def h_star_from_E(E_poly, d):

    t = var('t')
    h_coeff = []
    for k in range(d + 1):
        hk = sum((-1)^(k - j) * binomial(d + 1, k - j) * E_poly(j)
                 for j in range(k + 1))
        h_coeff.append(QQ(hk))
    # arma el polinomio h^*(t)
    h_poly = sum(h_coeff[k] * t^k for k in range(d + 1))
    return h_poly, h_coeff


def ehrhart_series(E_poly, h_poly, d):
 
    t = var('t')
    return h_poly / (1 - t)^(d + 1)

pol = []

for d in [2, 3, 4,5 ,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]:
    E = ehrhart_stasheff(d)
    h_poly, h_coeff = h_star_from_E(E, d)
    series = ehrhart_series(E, h_poly, d)
    print(f"\n─── Stasheff  d = {d} ───────────────────────────")
    print("E_d(n)      =", E)
    pol.append(E)


    print("h*(t)       =", h_poly.expand())
    print("coef h*     =", h_coeff)
    print("Ehrhart(t)  =", series.expand()) 
    
   


─── Stasheff  d = 2 ───────────────────────────
E_d(n)      = 7/2*n^2 + 7/2*n + 1
h*(t)       = t^2 + 5*t + 1
coef h*     = [1, 5, 1]
Ehrhart(t)  = -t^2/(t^3 - 3*t^2 + 3*t - 1) - 5*t/(t^3 - 3*t^2 + 3*t - 1) - 1/(t^3 - 3*t^2 + 3*t - 1)

─── Stasheff  d = 3 ───────────────────────────
E_d(n)      = 6*n^3 + 9*n^2 + 5*n + 1
h*(t)       = t^3 + 17*t^2 + 17*t + 1
coef h*     = [1, 17, 17, 1]
Ehrhart(t)  = t^3/(t^4 - 4*t^3 + 6*t^2 - 4*t + 1) + 17*t^2/(t^4 - 4*t^3 + 6*t^2 - 4*t + 1) + 17*t/(t^4 - 4*t^3 + 6*t^2 - 4*t + 1) + 1/(t^4 - 4*t^3 + 6*t^2 - 4*t + 1)

─── Stasheff  d = 4 ───────────────────────────
E_d(n)      = 41/4*n^4 + 41/2*n^3 + 67/4*n^2 + 13/2*n + 1
h*(t)       = t^4 + 50*t^3 + 144*t^2 + 50*t + 1
coef h*     = [1, 50, 144, 50, 1]
Ehrhart(t)  = -t^4/(t^5 - 5*t^4 + 10*t^3 - 10*t^2 + 5*t - 1) - 50*t^3/(t^5 - 5*t^4 + 10*t^3 - 10*t^2 + 5*t - 1) - 144*t^2/(t^5 - 5*t^4 + 10*t^3 - 10*t^2 + 5*t - 1) - 50*t/(t^5 - 5*t^4 + 10*t^3 - 10*t^2 + 5*t - 1) - 1/(t^5 - 5*t^4 + 10*t^3 - 10*t^2 + 5*t -

In [14]:
# Anillo de polinomios con coeficientes racionales
R.<s> = PolynomialRing(QQ)

# Campo complejo con 30 cifras decimales (~100 bits)
K = ComplexField(100)

for k in pol:
    Ehr_roots = k.roots(ring=K)      
    for z, mult in Ehr_roots:
        print(z)
    

-0.50000000000000000000000000000 - 0.18898223650461361360725826812*I
-0.50000000000000000000000000000 + 0.18898223650461361360725826812*I
-0.50000000000000000000000000000
-0.50000000000000000000000000000 - 0.28867513459481288225457439025*I
-0.50000000000000000000000000000 + 0.28867513459481288225457439025*I
-0.50000000000000000000000000000 - 0.34872828116652759412267359642*I
-0.50000000000000000000000000000 - 0.11195948989725663198859914124*I
-0.50000000000000000000000000000 + 0.11195948989725663198859914124*I
-0.50000000000000000000000000000 + 0.34872828116652759412267359642*I
-0.50000000000000000000000000000
-0.50000000000000000000000000000 - 0.38729833462074168851792653998*I
-0.50000000000000000000000000000 - 0.18898223650461361360725826812*I
-0.50000000000000000000000000000 + 0.18898223650461361360725826812*I
-0.50000000000000000000000000000 + 0.38729833462074168851792653998*I
-0.50000000000000000000000000000 - 0.41326149803512989546481171196*I
-0.50000000000000000000000000000 - 0.

-0.50000000000000000000000000000 - 0.48339367030326211371464139603*I
-0.50000000000000000000000000000 - 0.43850190462999834452762061028*I
-0.50000000000000000000000000000 - 0.37616827447041648241502563246*I
-0.50000000000000000000000000000 - 0.30644851407275017365859427189*I
-0.50000000000000000000000000000 - 0.23551691664831975019138793581*I
-0.50000000000000000000000000000 - 0.16605612485312467572613889144*I
-0.50000000000000000000000000000 - 0.098618515015468911199064500792*I
-0.50000000000000000000000000000 - 0.032691444782111983780029739743*I
-0.50000000000000000000000000000 + 0.032691444782111983780029739743*I
-0.50000000000000000000000000000 + 0.098618515015468911199064500792*I
-0.50000000000000000000000000000 + 0.16605612485312467572613889144*I
-0.50000000000000000000000000000 + 0.23551691664831975019138793581*I
-0.50000000000000000000000000000 + 0.30644851407275017365859427189*I
-0.50000000000000000000000000000 + 0.37616827447041648241502563246*I
-0.500000000000000000000000000

-0.50000000000000000000000000000 - 0.18898223650461361360725826812*I
-0.50000000000000000000000000000 - 0.13370616487128252561341647541*I
-0.50000000000000000000000000000 - 0.079665380367832667023984278335*I
-0.50000000000000000000000000000 - 0.026458043099385824648131661170*I
-0.50000000000000000000000000000 + 0.026458043099385824648131661170*I
-0.50000000000000000000000000000 + 0.079665380367832667023984278335*I
-0.50000000000000000000000000000 + 0.13370616487128252561341647541*I
-0.50000000000000000000000000000 + 0.18898223650461361360725826812*I
-0.50000000000000000000000000000 + 0.24559328198747364250336907738*I
-0.50000000000000000000000000000 + 0.30306566539992172457843729233*I
-0.50000000000000000000000000000 + 0.35993855566924244241839340266*I
-0.50000000000000000000000000000 + 0.41326149803512989546481171196*I
-0.50000000000000000000000000000 + 0.45829262146458771234162489026*I
-0.50000000000000000000000000000 + 0.48901386678768369965047267193*I
